### Remake coordinates_bdy.nc file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

#### Functions

In [73]:
def make_bdy_coord(t_coords, u_coords, v_coords, domain_cfg, 
                   rimwidth=1, filename='/home/users/birgal/data/NEMO/coordinates_bdy_new.nc'):
    """ Function makes a NEMO coordinate_bdy NetCDF file given input coordinates and a domain
    Arguments
    ----------
        t_coords : list
            Tuples of i,j coordinates of gridT points in the boundary. Assumes the order goes from the inside of the domain to the outside.
        u_coords : list
            Same as above but for the grid U points
        v_coords : list
            Same as above but for the grid V points
        domain_cfg : xarray dataset 
            From the domain_cfg NEMO file

    Parameters
    ----------
        filename : string 
            Specifies output file location and name
        rimwidth : integer 
            Width of your NEMO boundary (typically between 8-10 gridpoints)
            
    Returns
    ----------
        Dataset : xarray dataset
    """

    it_coords, jt_coords = np.array(list(zip(*t_coords))).astype(int)
    iu_coords, ju_coords = np.array(list(zip(*u_coords))).astype(int)
    iv_coords, jv_coords = np.array(list(zip(*v_coords))).astype(int)

    # Take subsets of the domain_cfg dataset based on the coordinates of boundary points
    domain_cfg_subset_t = domain_cfg.sel(x=xr.DataArray(it_coords, dims='z'), y=xr.DataArray(jt_coords, dims='z'))
    domain_cfg_subset_u = domain_cfg.sel(x=xr.DataArray(iu_coords, dims='z'), y=xr.DataArray(ju_coords, dims='z'))
    domain_cfg_subset_v = domain_cfg.sel(x=xr.DataArray(iv_coords, dims='z'), y=xr.DataArray(jv_coords, dims='z'))
    
    # Dimensions:
    bdy_length = int(np.floor(len(t_coords)/rimwidth))
    yb_dim     = 1 # always
    xbv_dim    = (bdy_length)  *rimwidth
    xbu_dim    = (bdy_length-1)*rimwidth
    xbt_dim    = (bdy_length)  *rimwidth
                       
    # Create xarray dataset:
    ds = xr.Dataset(
        data_vars=dict(
                    e1t=(["yb", "xbt"], domain_cfg_subset_t['e1t'].values.astype(float)),
                    e1u=(["yb", "xbu"], domain_cfg_subset_u['e1u'].values.astype(float)), 
                    e1v=(["yb", "xbv"], domain_cfg_subset_v['e1v'].values.astype(float)), 
                    e2t=(["yb", "xbt"], domain_cfg_subset_t['e2t'].values.astype(float)),
                    e2u=(["yb", "xbu"], domain_cfg_subset_u['e2u'].values.astype(float)),
                    e2v=(["yb", "xbv"], domain_cfg_subset_v['e2v'].values.astype(float)),
                    glamt=(["yb", "xbt"], domain_cfg_subset_t['glamt'].values.astype(float)),
                    glamu=(["yb", "xbu"], domain_cfg_subset_u['glamu'].values.astype(float)),
                    glamv=(["yb", "xbv"], domain_cfg_subset_v['glamv'].values.astype(float)),
                    gphit=(["yb", "xbt"], domain_cfg_subset_t['gphit'].values.astype(float)),
                    gphiu=(["yb", "xbu"], domain_cfg_subset_u['gphiu'].values.astype(float)),
                    gphiv=(["yb", "xbv"], domain_cfg_subset_v['gphiv'].values.astype(float)),                
                    nbit=(["yb", "xbt"], np.reshape(it_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbiu=(["yb", "xbu"], np.reshape(iu_coords+1, (yb_dim, xbu_dim)).astype(int)),
                    nbiv=(["yb", "xbv"], np.reshape(iv_coords+1, (yb_dim, xbv_dim)).astype(int)),
                    nbjt=(["yb", "xbt"], np.reshape(jt_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbju=(["yb", "xbu"], np.reshape(ju_coords+1, (yb_dim, xbu_dim)).astype(int)),
                    nbjv=(["yb", "xbv"], np.reshape(jv_coords+1, (yb_dim, xbv_dim)).astype(int)),
                    nbrt=(["yb", "xbt"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length)  , (yb_dim, xbt_dim)).astype(int)),
                    nbru=(["yb", "xbu"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length-1), (yb_dim, xbu_dim)).astype(int)),
                    nbrv=(["yb", "xbv"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length)  , (yb_dim, xbv_dim)).astype(int)),
        ),
        attrs=dict(description="Boundary condition coordinates file"),
    )
    
    ds.to_netcdf(f'{filename}')


    return ds

In [100]:
def make_bdy_seaice(t_coords, ice_bdy, domain_cfg, rimwidth=1, 
                    filename='/home/users/birgal/data/boundary_conditions/bdyT_ice_y1979_test.nc'):
    """ Function makes a NEMO ice boundary forcing NetCDF file given input coordinates
    Arguments
    ----------
        t_coords : list
            Tuples of i,j coordinates of gridT points in the boundary. Assumes the order goes from the inside of the domain to the outside.
        u_coords : list
        ice_bdy : xarray dataset 
            A sea ice boundary condition file to use to get the timestamps etc.

    Parameters
    ----------
        filename : string 
            Specifies output file location and name
        rimwidth : integer 
            Width of your NEMO boundary (typically between 8-10 gridpoints)
            
    Returns
    ----------
        Dataset : xarray dataset
    """

    it_coords, jt_coords = np.array(list(zip(*t_coords))).astype(int)

    # Take subsets of the domain_cfg dataset based on the coordinates of boundary points
    domain_cfg_subset = domain_cfg.sel(x=xr.DataArray(it_coords, dims='z'), y=xr.DataArray(jt_coords, dims='z'))
    
    # Dimensions:
    bdy_length = int(np.floor(len(t_coords)/rimwidth))
    yb_dim     = 1  # always
    time_dim   = 73 # five-day frequency
    xbt_dim    = bdy_length*rimwidth
                       
    # Create xarray dataset:
    ds = xr.Dataset(
        data_vars=dict(
                    time_counter_bnds=(["time_counter", "nbds"], ice_bdy['time_counter_bnds'].values), 
                    nbidta=(["yb", "xbT"], np.reshape(it_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbjdta=(["yb", "xbT"], np.reshape(jt_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbrdta=(["yb", "xbT"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length), (yb_dim, xbt_dim)).astype(int)),
                    siconc=(["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xbt_dim))),
                    sithic=(["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xbt_dim))),
                    sivolu=(["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xbt_dim))),
                    snthic=(["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xbt_dim))),
                    snvolu=(["time_counter", "yb", "xbT"], np.zeros((time_dim, yb_dim, xbt_dim))),
        ),
        coords=dict(
                time_counter=ice_bdy['time_counter'].values,
                nav_lon=(["yb", "xbT"], np.reshape(domain_cfg_subset['nav_lon'].values, (yb_dim, xbt_dim))),
                nav_lat=(["yb", "xbT"], np.reshape(domain_cfg_subset['nav_lat'].values, (yb_dim, xbt_dim))),
        ),
        attrs=dict(description="Sea ice boundary condition file"),
    )
    
    ds.to_netcdf(f'{filename}')


    return ds

In [125]:
def make_bdy_tracer(t_coords, tra_bdy, tra_out, domain_cfg, rimwidth=1, 
                    filename='/home/users/birgal/data/boundary_conditions/bdyT_tra_y1979_test.nc'):
    """ Function makes a NEMO tracer boundary forcing NetCDF file given input coordinates
    Arguments
    ----------
        t_coords : list
            Tuples of i,j coordinates of gridT points in the boundary. Assumes the order goes from the inside of the domain to the outside.
        tra_bdy : xarray dataset 
            A tracer boundary condition file to use to get the timestamps etc.

    Parameters
    ----------
        filename : string 
            Specifies output file location and name
        rimwidth : integer 
            Width of your NEMO boundary (typically between 8-10 gridpoints)
            
    Returns
    ----------
        Dataset : xarray dataset
    """

    it_coords, jt_coords = np.array(list(zip(*t_coords))).astype(int)

    # Take subsets of the domain_cfg dataset based on the coordinates of boundary points
    domain_cfg_subset = domain_cfg.sel(x=xr.DataArray(it_coords, dims='z'), y=xr.DataArray(jt_coords, dims='z'))
    tra_slice         = tra_out.sel(time_counter=tra_out['time_counter'][0],
                                    x_grid_T=xr.DataArray(it_coords-1, dims='z'), 
                                    y_grid_T=xr.DataArray(jt_coords-1, dims='z'))
    
    # Dimensions:
    bdy_length = int(np.floor(len(t_coords)/rimwidth))
    depth_dim  = 121
    yb_dim     = 1  # always
    time_dim   = 73 # five-day frequency
    xbt_dim    = bdy_length*rimwidth

    # Create variables:
    sossh_fill    = np.empty((time_dim, yb_dim, xbt_dim));
    vosaline_fill = np.empty((time_dim, depth_dim, yb_dim, xbt_dim));
    votemper_fill = np.empty((time_dim, depth_dim, yb_dim, xbt_dim));
    sossh_fill[:]    = np.reshape(tra_slice['zos'].values, (yb_dim, xbt_dim))
    vosaline_fill[:] = np.reshape(tra_slice['so'].values, (depth_dim, yb_dim, xbt_dim))
    votemper_fill[:] = np.reshape(tra_slice['thetao'].values, (depth_dim, yb_dim, xbt_dim))
    vosaline_fill[vosaline_fill < 10] = 10
                        
    # Create xarray dataset:
    ds = xr.Dataset(
        data_vars=dict(
                    time_counter_bnds=(["time_counter", "nbds"], tra_bdy['time_counter_bnds'].values), 
                    deptht_bnds = (["deptht","bnds"], tra_bdy['deptht_bnds'].values), 
                    nbidta   = (["yb", "xbT"], np.reshape(it_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbjdta   = (["yb", "xbT"], np.reshape(jt_coords+1, (yb_dim, xbt_dim)).astype(int)),
                    nbrdta   = (["yb", "xbT"], np.reshape(np.repeat(np.arange(1,rimwidth+1), bdy_length), (yb_dim, xbt_dim)).astype(int)),
                    sossh    = (["time_counter", "yb", "xbT"]          , sossh_fill),
                    votemper = (["time_counter", "deptht", "yb", "xbT"], votemper_fill),
                    vosaline = (["time_counter", "deptht", "yb", "xbT"], vosaline_fill), 
        ),
        coords=dict(
                time_counter = tra_bdy['time_counter'].values,
                nav_lon      = (["yb", "xbT"], np.reshape(domain_cfg_subset['nav_lon'].values, (yb_dim, xbt_dim))),
                nav_lat      = (["yb", "xbT"], np.reshape(domain_cfg_subset['nav_lat'].values, (yb_dim, xbt_dim))),
                deptht       = tra_bdy['deptht'].values,
        ),
        attrs=dict(description="Tracer boundary condition file"),
    )
    
    ds.to_netcdf(f'{filename}')


    return ds

### Read files
- Use coordinates from the domain_cfg.nc file or from the old boundary file

In [27]:
coord_bdy_old = xr.open_dataset('/home/users/birgal/data/NEMO/coordinates_bdy.nc')
domain_cfg    = xr.open_dataset('/home/users/birgal/data/NEMO/domain_cfg.nc')

### Create coordinates boundary file

In [86]:
# Python coordinates of the boundary (zero-based). Assumes you go from the inside of the boundary to the outside
# T grid boundary coordinates
it_coords = (domain_cfg.x.values).astype(int)                            # indices along northern boundary
jt_coords = np.tile(domain_cfg.y.values[-1], len(it_coords)).astype(int) # boundary along constant j index at edge of domain
# U grid boundary coordinates
iu_coords = np.copy(it_coords)[:-1] # one less grid point on the east edge (I think)
ju_coords = np.copy(jt_coords)[:-1] # same as "jt" except one gridpoint shorter
# V grid boundary coordinates
iv_coords = np.copy(it_coords)   # same as "it"
jv_coords = np.copy(jt_coords)-1 # "jt - 1" for northern boundary

# Create coordinate pairs:
T_coords = list(zip(it_coords, jt_coords))
U_coords = list(zip(iu_coords, ju_coords))
V_coords = list(zip(iv_coords, jv_coords))

In [89]:
ds_coord = make_bdy_coord(T_coords, U_coords, V_coords, domain_cfg, rimwidth=1, \
                          filename='/home/users/birgal/data/NEMO/coordinates_bdy_test.nc')

### Create ice boundary file

In [101]:
# Use the same coordinates as above
ice_out = xr.open_dataset('/home/users/birgal/data/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_icemod.nc')
ice_bdy = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyT_ice_y1979.nc')

ds_ice  = make_bdy_seaice(T_coords, ice_bdy, domain_cfg, rimwidth=1, \
                          filename='/home/users/birgal/data/boundary_conditions/bdyT_ice_y1979_test.nc')

### Create tracer boundary file

In [126]:
tra_out   = xr.open_dataset('/home/users/birgal/data/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_T.nc')
tra_bdy   = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyT_tra_y1979.nc')

ds_tra    = make_bdy_tracer(T_coords, tra_bdy, tra_out, domain_cfg, rimwidth=1, 
                            filename='/home/users/birgal/data/boundary_conditions/bdyT_tra_y1979_test.nc')

### Create U, V boundary files

In [118]:
gridU_bdy   = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyU_u3d_y1979.nc')
gridU_out   = xr.open_dataset('/home/users/birgal/data/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_U.nc')
gridU_slice = gridU_out.sel(time_counter=gridU_out['time_counter'][0], 
                            x=xr.DataArray(iu_coords_rw2, dims='z'), 
                            y=xr.DataArray(ju_coords_rw2, dims='z'))

In [119]:
# Dimensions:
rimwidth  = 2
time_dim  = 73
depth_dim = 121
xbu_dim   = (1424-1)*rimwidth
yb_dim    = 1

# Create variables of the right dimensions:
vozocrtx_fill    = np.zeros((time_dim, depth_dim, yb_dim, xbu_dim));
vozocrtx_fill[:] = np.reshape(gridU_slice['uo2'].values, (depth_dim, yb_dim, xbu_dim))

# Create xarray dataset: 
ds_gridU = xr.Dataset(
    data_vars=dict(
                time_counter_bnds=(["time_counter", "bnds"], gridU_bdy['time_counter_bnds'].values),
                depthu_bnds=(["depthu","bnds"], gridU_bdy['depthu_bnds'].values),
                nbidta=(["yb", "xbU"], np.reshape(iu_coords_rw2+1, (yb_dim, xbu_dim)).astype(int)),
                nbjdta=(["yb", "xbU"], np.reshape(ju_coords_rw2+1, (yb_dim, xbu_dim)).astype(int)),
                nbrdta=(["yb", "xbU"], np.reshape(np.append(np.ones(bdy_length-1), 
                                                            np.ones((yb_dim, bdy_length-1))*2), (yb_dim, xbu_dim)).astype(int)),
                vozocrtx=(["time_counter", "deptht", "yb", "xbU"], vozocrtx_fill), 
    ),
    coords=dict(
                time_counter = gridU_bdy['time_counter'].values,
                nav_lon      = (["yb", "xbU"], np.reshape(gridU_slice['nav_lon'].values, (yb_dim, xbu_dim))),
                nav_lat      = (["yb", "xbU"], np.reshape(gridU_slice['nav_lat'].values, (yb_dim, xbu_dim))),
                depthu       = gridU_bdy['depthu'].values,
    ),
    attrs=dict(description   = "Test gridU boundary condition file"),
)

ds_gridU.to_netcdf(f'/home/users/birgal/data/boundary_conditions/rimwidth2/bdyU_u3d_y1979.nc')

In [120]:
gridV_bdy   = xr.open_dataset('/home/users/birgal/data/forcing_NEMO/bdyV_u3d_y1979.nc')
gridV_out   = xr.open_dataset('/home/users/birgal/data/NEMO-test3-20230914/eANT025.L121_1m_19790101_19791231_grid_V.nc')
gridV_slice = gridV_out.sel(time_counter=gridV_out['time_counter'][0], 
                            x=xr.DataArray(iv_coords_rw2, dims='z'), 
                            y=xr.DataArray(jv_coords_rw2, dims='z'))

In [121]:
# Dimensions:
rimwidth  = 2
time_dim  = 73
depth_dim = 121
xbv_dim   = 1424*rimwidth
yb_dim    = 1

# Create variables of the right dimensions:
vomecrty_fill    = np.zeros((time_dim, depth_dim, yb_dim, xbv_dim));
vomecrty_fill[:] = np.reshape(gridV_slice['vo2'].values, (depth_dim, yb_dim, xbv_dim))

# Create xarray dataset: 
ds_gridV = xr.Dataset(
    data_vars=dict(
                time_counter_bnds=(["time_counter", "bnds"], gridV_bdy['time_counter_bnds'].values),
                depthv_bnds=(["depthv","bnds"], gridV_bdy['depthv_bnds'].values),
                nbidta=(["yb", "xbV"], np.reshape(iv_coords_rw2+1, (yb_dim, xbv_dim)).astype(int)),
                nbjdta=(["yb", "xbV"], np.reshape(jv_coords_rw2+1, (yb_dim, xbv_dim)).astype(int)),
                nbrdta=(["yb", "xbV"], np.reshape(np.append(np.ones(bdy_length), 
                                                            np.ones((yb_dim, bdy_length))*2), (yb_dim, xbv_dim)).astype(int)),
                vomecrty=(["time_counter", "deptht", "yb", "xbV"], vomecrty_fill), 
    ),
    coords=dict(
                time_counter = gridV_bdy['time_counter'].values,
                nav_lon      = (["yb", "xbV"], np.reshape(gridV_slice['nav_lon'].values, (yb_dim, xbv_dim))),
                nav_lat      = (["yb", "xbV"], np.reshape(gridV_slice['nav_lat'].values, (yb_dim, xbv_dim))),
                depthv       = gridV_bdy['depthv'].values,
    ),
    attrs=dict(description="Test gridV boundary condition file"),
)

ds_gridV.to_netcdf(f'/home/users/birgal/data/boundary_conditions/rimwidth2/bdyV_u3d_y1979.nc')